# Events and Listeners

As already mentioned in the user guide before, the {class}`~maverick.game.Game` class is implemented as a state machine and the event-driven execution model triggers the state transitions. These events are emitted to registered listeners.

## Events

The different types of events a listener can listen to are encapsulated by the {class}`~maverick.enums.GameEventType` enumeration class and you can sign up to any of the events listed there.

## Listening to Events

### Explicit Event Listeners

One way of defining listeners is to define and register Python functions. The protocol a listener has to adhere the following function signature:

```python
def custom_listener(event: GameEvent, game: Game) -> None:
    # Do something here
    ...
```

Since the game itself is passed to the listeners, the listeners can even interfere with the game. To register an event handler, call the `subscribe` method of the game instance, where the first parameter is the type of event you want to subscribe to, and the second is the listener itself.

In [4]:
from maverick import Game, GameEvent, GameEventType, PlayerState
from maverick.players import FoldBot, CallBot, AggressiveBot

print("=" * 60)
print("Maverick Event System Demo")
print("=" * 60)
print()

# Create a game
game = Game(small_blind=10, big_blind=20, max_hands=1)

# Create a global event counter
event_counts = {event_type: 0 for event_type in GameEventType}


def count_events(event: GameEvent, game: Game) -> None:
    """Count each type of event."""
    event_counts[event.type] += 1


def log_major_events(event: GameEvent, game: Game) -> None:
    """Log major game events."""
    if event.type in [
        GameEventType.GAME_STARTED,
        GameEventType.HAND_STARTED,
        GameEventType.HAND_ENDED,
    ]:
        print(f"📢 {event.type.name} (Hand #{event.hand_number}, Street: {event.street.name})")


def log_player_actions(event: GameEvent, game: Game) -> None:
    """Log player actions with details."""
    if event.type == GameEventType.PLAYER_ACTION:
        action_str = f"{event.action.action_type.name}"
        if event.action.amount:
            action_str += f" ({event.action.amount} chips)"
        print(
            f"  🎲 Player {event.player_id}: {action_str} | "
            f"Pot: {game.state.pot} | Current Bet: {game.state.current_bet}"
        )


def log_street_changes(event: GameEvent, game: Game) -> None:
    """Log when streets change."""
    if event.type in [
        GameEventType.DEAL_FLOP,
        GameEventType.DEAL_TURN,
        GameEventType.DEAL_RIVER,
    ]:
        print(f"  🃏 {event.type.name} - Moving to {event.street.name}")


# Register event handlers
game.subscribe(GameEventType.GAME_STARTED, count_events)
game.subscribe(GameEventType.GAME_STARTED, log_major_events)
game.subscribe(GameEventType.GAME_ENDED, count_events)
game.subscribe(GameEventType.GAME_ENDED, log_major_events)
game.subscribe(GameEventType.HAND_STARTED, count_events)
game.subscribe(GameEventType.HAND_STARTED, log_major_events)
game.subscribe(GameEventType.HAND_ENDED, count_events)
game.subscribe(GameEventType.HAND_ENDED, log_major_events)
game.subscribe(GameEventType.PLAYER_ACTION, count_events)
game.subscribe(GameEventType.PLAYER_ACTION, log_player_actions)
game.subscribe(GameEventType.DEAL_HOLE_CARDS, count_events)
game.subscribe(GameEventType.DEAL_HOLE_CARDS, log_player_actions)
game.subscribe(GameEventType.BETTING_ROUND_COMPLETED, count_events)
game.subscribe(GameEventType.BETTING_ROUND_COMPLETED, log_player_actions)
game.subscribe(GameEventType.POST_BLINDS, count_events)
game.subscribe(GameEventType.POST_BLINDS, log_player_actions)
game.subscribe(GameEventType.SHOWDOWN, count_events)
game.subscribe(GameEventType.SHOWDOWN, log_player_actions)
game.subscribe(GameEventType.PLAYER_JOINED, count_events)
game.subscribe(GameEventType.PLAYER_JOINED, log_player_actions)
game.subscribe(GameEventType.PLAYER_LEFT, count_events)
game.subscribe(GameEventType.PLAYER_LEFT, log_player_actions)
game.subscribe(GameEventType.DEAL_FLOP, count_events)
game.subscribe(GameEventType.DEAL_FLOP, log_street_changes)
game.subscribe(GameEventType.DEAL_TURN, count_events)
game.subscribe(GameEventType.DEAL_TURN, log_street_changes)
game.subscribe(GameEventType.DEAL_RIVER, count_events)
game.subscribe(GameEventType.DEAL_RIVER, log_street_changes)


class EventRecorder:
    """Helper class to record events in order."""

    def __init__(self):
        self.events: list[GameEvent] = []

    def record(self, event: GameEvent, game: Game) -> None:
        self.events.append(event)

    def clear(self) -> None:
        self.events = []

    def get_event_types(self) -> list[GameEventType]:
        return [e.type for e in self.events]
    

recorder = EventRecorder()
game.subscribe(GameEventType.GAME_STARTED, recorder.record)

# Add players (one with event hook)
p1 = FoldBot(name="Alice", state=PlayerState(stack=1000))
p2 = AggressiveBot(name="Bob", state=PlayerState(stack=1000))
p3 = CallBot(name="Charlie", state=PlayerState(stack=1000))

# Add the players to the game
game.add_player(p1)
game.add_player(p2)
game.add_player(p3)

print(f"Added 3 players: {p1.name}, {p2.name}, {p3.name}")
print()

# Run the game
print("Starting game...")
print()
game.start()

game.remove_player(p1)
game.remove_player(p2)
game.remove_player(p3)

print()
print("=" * 60)
print("Event Summary")
print("=" * 60)
print()

# Print event counts
print("Global Event Counts:")
for event_type, count in sorted(event_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"  {event_type.name:30s}: {count:3d}")

print()
print("=" * 60)
print("Demo Complete!")
print("=" * 60)

Maverick Event System Demo

Added 3 players: Alice, Bob, Charlie

Starting game...

📢 GAME_STARTED (Hand #0, Street: PRE_FLOP)
📢 HAND_STARTED (Hand #1, Street: PRE_FLOP)
  🎲 Player cf554285b1df4e24907c741a71a69607: FOLD | Pot: 30 | Current Bet: 20
  🎲 Player e632e385a15b42c7ae6d325e8030bc31: RAISE (30 chips) | Pot: 60 | Current Bet: 40
  🎲 Player 200ec6ec9c204fb28b12685293f418dd: CALL | Pot: 80 | Current Bet: 40
  🃏 DEAL_FLOP - Moving to FLOP
  🎲 Player e632e385a15b42c7ae6d325e8030bc31: BET (40 chips) | Pot: 120 | Current Bet: 40
  🎲 Player 200ec6ec9c204fb28b12685293f418dd: CALL | Pot: 160 | Current Bet: 40
  🃏 DEAL_TURN - Moving to TURN
  🎲 Player e632e385a15b42c7ae6d325e8030bc31: BET (40 chips) | Pot: 200 | Current Bet: 40
  🎲 Player 200ec6ec9c204fb28b12685293f418dd: CALL | Pot: 240 | Current Bet: 40
  🃏 DEAL_RIVER - Moving to RIVER
  🎲 Player e632e385a15b42c7ae6d325e8030bc31: BET (40 chips) | Pot: 280 | Current Bet: 40
  🎲 Player 200ec6ec9c204fb28b12685293f418dd: CALL | Pot: 320 | C

## Implicit Event Listeners

There is another way of subscribing to certain events, but it is only available for players at the moment.

In [5]:
from maverick import Game, GameEvent
from collections import defaultdict


class ListenerCallBot(CallBot):
    """A player that counts the observed events by type."""

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.event_counts = defaultdict(int)

    def on_event(self, event: GameEvent, game: Game) -> None:
        """Track how many events this player observes."""
        self.event_counts[event.type] += 1
        print(f"Player {self.name} observed event: {event.type.name}")
        
    def on_game_ended(self, event: GameEvent, game: Game) -> None:
        """Report the event counts when the game ends."""
        assert event.type == GameEventType.GAME_ENDED
        print(f"{self.name}: Bye!")


game = Game(small_blind=10, big_blind=20, max_hands=1)

# Add players (one with event hook)
p1 = FoldBot(name="Alice", state=PlayerState(stack=1000))
p2 = ListenerCallBot(name="Bob", state=PlayerState(stack=1000))
p3 = CallBot(name="Charlie", state=PlayerState(stack=1000))

# Add the players to the game
game.add_player(p1)
game.add_player(p2)
game.add_player(p3)

# Run the game
game.start()

Player Bob observed event: PLAYER_JOINED
Player Bob observed event: PLAYER_JOINED
Player Bob observed event: GAME_STARTED
Player Bob observed event: HAND_STARTED
Player Bob observed event: DEAL_HOLE_CARDS
Player Bob observed event: POST_BLINDS
Player Bob observed event: POST_ANTES
Player Bob observed event: PLAYER_ACTION
Player Bob observed event: PLAYER_ACTION
Player Bob observed event: PLAYER_ACTION
Player Bob observed event: BETTING_ROUND_COMPLETED
Player Bob observed event: DEAL_FLOP
Player Bob observed event: PLAYER_ACTION
Player Bob observed event: PLAYER_ACTION
Player Bob observed event: BETTING_ROUND_COMPLETED
Player Bob observed event: DEAL_TURN
Player Bob observed event: PLAYER_ACTION
Player Bob observed event: PLAYER_ACTION
Player Bob observed event: BETTING_ROUND_COMPLETED
Player Bob observed event: DEAL_RIVER
Player Bob observed event: PLAYER_ACTION
Player Bob observed event: PLAYER_ACTION
Player Bob observed event: BETTING_ROUND_COMPLETED
Player Bob observed event: SHOWDO

In [6]:
p2.event_counts

defaultdict(int,
            {<GameEventType.PLAYER_JOINED: 14>: 2,
             <GameEventType.GAME_STARTED: 1>: 1,
             <GameEventType.HAND_STARTED: 2>: 1,
             <GameEventType.DEAL_HOLE_CARDS: 5>: 1,
             <GameEventType.POST_BLINDS: 11>: 1,
             <GameEventType.POST_ANTES: 12>: 1,
             <GameEventType.PLAYER_ACTION: 9>: 9,
             <GameEventType.BETTING_ROUND_COMPLETED: 10>: 4,
             <GameEventType.DEAL_FLOP: 6>: 1,
             <GameEventType.DEAL_TURN: 7>: 1,
             <GameEventType.DEAL_RIVER: 8>: 1,
             <GameEventType.SHOWDOWN: 13>: 1,
             <GameEventType.HAND_ENDED: 3>: 1,
             <GameEventType.GAME_ENDED: 4>: 1})